In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
import pandas as pd

# Read the CSV and Perform Basic Data Cleaning

In [ ]:
df = pd.read_csv("exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

# Select your features (columns)

In [ ]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_disposition', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_period', 'koi_fpflag_co', 'koi_prad', 'koi_steff', 'koi_duration', 
                        'koi_time0bk', 'koi_kepmag', 'koi_model_snr', 'koi_duration_err2', 'koi_duration_err1', 'koi_prad_err1', 'koi_steff_err1', 
                        'koi_fpflag_ec', 'koi_prad_err2', 'koi_steff_err2', 'koi_time0bk_err1', 'koi_depth', 'koi_time0bk_err2', 'koi_period_err2', 
                        'koi_insol_err1', 'koi_impact', 'koi_period_err1', 'koi_insol_err2', 'koi_insol']]

# Create a Train Test Split

Use `koi_disposition` for the y values

In [ ]:
X = selected_features.drop("koi_disposition", axis=1)
y = selected_features[['koi_disposition']]
print(X.shape, y.shape)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

X_train.head()

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [ ]:
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
y_train_encoded = label_encoder.transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


# Scale your data
X_scaler = MinMaxScaler().fit(X)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train_categorical = to_categorical(y_train_encoded)
y_test_categorical = to_categorical(y_test_encoded)

# Train the Model



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(units=120, activation='relu', input_dim=40))
model.add(Dense(units=120, activation='relu'))
model.add(Dense(units=3, activation='softmax'))

In [ ]:
# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# Fit the model to the training data
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=100,
    shuffle=True,
    verbose=2
)

In [ ]:
print(f"Training Data Score: {model.score(X_train_scaled, y_train_encoded)}")
print(f"Testing Data Score: {model.score(X_test_scaled, y_test_encoded)}")

# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [ ]:
# Create the GridSearchCV model
# from sklearn.svm import SVC 
# model2 = SVC(kernel='linear')

from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10, 50],
              'max_iter': [500, 1000, 5000]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [ ]:
# Train the model with GridSearch
grid.fit(X_train_scaled, y_train_encoded)

In [ ]:
print(grid.best_params_)
print(grid.best_score_)

# Save the Model

In [ ]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash

# import joblib
# filename = 'TylerNguyen.sav'
# joblib.dump(LogisticRegression, filename)